### - взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
print(df.shape)
df.head(3)

/home/mr-rider/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


(7500, 17)


,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0


### - сделать обзорный анализ выбранного датасета

In [2]:
df.shape

(7500, 17)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Home Ownership                7500 non-null   object 
 1   Annual Income                 5943 non-null   float64
 2   Years in current job          7129 non-null   object 
 3   Tax Liens                     7500 non-null   float64
 4   Number of Open Accounts       7500 non-null   float64
 5   Years of Credit History       7500 non-null   float64
 6   Maximum Open Credit           7500 non-null   float64
 7   Number of Credit Problems     7500 non-null   float64
 8   Months since last delinquent  3419 non-null   float64
 9   Bankruptcies                  7486 non-null   float64
 10  Purpose                       7500 non-null   object 
 11  Term                          7500 non-null   object 
 12  Current Loan Amount           7500 non-null   float64
 13  Cur

In [4]:
# Анализ целевой переменной
df['Credit Default'].value_counts()

0    5387
1    2113
Name: Credit Default, dtype: int64

In [5]:
df.describe()

,Annual Income,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
count,5.943000e+03,7500.000000,7500.000000,7500.000000,7.500000e+03,7500.000000,3419.000000,7486.000000,7.500000e+03,7.500000e+03,7500.000000,5943.000000,7500.000000
mean,1.366392e+06,0.030133,11.130933,18.317467,9.451537e+05,0.170000,34.692600,0.117152,1.187318e+07,2.898332e+05,18314.454133,1151.087498,0.281733
std,8.453392e+05,0.271604,4.908924,7.041946,1.602622e+07,0.498598,21.688806,0.347192,3.192612e+07,3.178714e+05,11926.764673,1604.451418,0.449874
min,1.645970e+05,0.000000,2.000000,4.000000,0.000000e+00,0.000000,0.000000,0.000000,1.124200e+04,0.000000e+00,0.000000,585.000000,0.000000
25%,8.443410e+05,0.000000,8.000000,13.500000,2.792295e+05,0.000000,16.000000,0.000000,1.801690e+05,1.142565e+05,10067.500000,711.000000,0.000000
50%,1.168386e+06,0.000000,10.000000,17.000000,4.781590e+05,0.000000,32.000000,0.000000,3.095730e+05,2.093230e+05,16076.500000,731.000000,0.000000
75%,1.640137e+06,0.000000,14.000000,21.800000,7.935015e+05,0.000000,50.000000,0.000000,5.198820e+05,3.604062e+05,23818.000000,743.000000,1.000000
max,1.014934e+07,7.000000,43.000000,57.700000,1.304726e+09,7.000000,118.000000,4.000000,1.000000e+08,6.506797e+06,136679.000000,7510.000000,1.000000


In [6]:
# Убираем лишние 0 в Credit Score  в train
for cr_score in df['Credit Score']:
    if (len(str(cr_score))) > 5:
        # print(cr_score)
        df.loc[df['Credit Score'] == cr_score, 'Credit Score'] = cr_score/10

In [7]:
# Убираем лишние 0 в Credit Score в test
for cr_score in df_test['Credit Score']:
    if (len(str(cr_score))) > 5:
        # print(cr_score)
        df_test.loc[df_test['Credit Score'] == cr_score, 'Credit Score'] = cr_score/10

In [8]:
df = df[df['Maximum Open Credit']<99999999]
df_test = df_test[df_test['Maximum Open Credit']<99999999]

In [9]:
median_loan_by_purpose = df.groupby('Purpose', as_index=False).agg({'Current Loan Amount':'median'})
median_loan_by_purpose_test = df_test.groupby('Purpose', as_index=False).agg({'Current Loan Amount':'median'})

In [10]:
for purpose in median_loan_by_purpose['Purpose']:
    df.loc[(df['Current Loan Amount']==99999999) & (df['Purpose']==str(purpose)), 'Current Loan Amount'] = median_loan_by_purpose.loc[median_loan_by_purpose['Purpose']==purpose, 'Current Loan Amount']

for purpose in median_loan_by_purpose_test['Purpose']:
    df_test.loc[(df_test['Current Loan Amount']==99999999) & (df_test['Purpose']==str(purpose)), 'Current Loan Amount'] = median_loan_by_purpose_test.loc[median_loan_by_purpose_test['Purpose']==purpose, 'Current Loan Amount']

##### Обработка пропусков

In [11]:
median_annual_income = df['Annual Income'].median()
df['Annual Income'] = df['Annual Income'].fillna(median_annual_income)


In [12]:
median_annual_income = df_test['Annual Income'].median()
df_test['Annual Income'] = df_test['Annual Income'].fillna(median_annual_income)


In [13]:
median_cred_score = df['Credit Score'].median()
df['Credit Score'] = df['Credit Score'].fillna(median_cred_score)


In [14]:
median_cred_score = df_test['Credit Score'].median()
df_test['Credit Score'] = df_test['Credit Score'].fillna(median_cred_score)

In [15]:
df['Bankruptcies'] = df['Bankruptcies'].fillna(0)
df_test['Bankruptcies'] = df_test['Bankruptcies'].fillna(0)

In [16]:
df.loc[df['Years in current job'].isnull(), 'Years in current job'] = df['Years in current job'].mode()[0]
df_test.loc[df_test['Years in current job'].isnull(), 'Years in current job'] = df_test['Years in current job'].mode()[0]

In [17]:
df['Months since last delinquent'] = df['Months since last delinquent'].fillna(0)
df_test['Months since last delinquent'] = df_test['Months since last delinquent'].fillna(0)


In [18]:
# Сохраним подготовленные данные
df_train_prep= df.copy()
df_test_prep=df_test.copy()

##### Обучение модели

In [19]:
# Целевой признак
target = 'Credit Default'
# Все признаки 
features_all = df_train_prep.columns.drop('Credit Default').tolist()
# Num
features_without_categ =['Annual Income','Tax Liens','Number of Open Accounts','Years of Credit History','Maximum Open Credit', 'Number of Credit Problems','Bankruptcies', 'Current Loan Amount', 'Current Credit Balance','Monthly Debt','Credit Score']
features_categ=['Home Ownership','Years in current job','Purpose','Term']

In [20]:
for colname in features_categ:
    df_train_prep[colname] = pd.Categorical(df_train_prep[colname])
    
df_train_prep[features_categ].dtypes

Home Ownership          category
Years in current job    category
Purpose                 category
Term                    category
dtype: object

In [21]:
for colname in features_categ:
    df_test_prep[colname] = pd.Categorical(df_test_prep[colname])
    
df_test_prep[features_categ].dtypes

Home Ownership          category
Years in current job    category
Purpose                 category
Term                    category
dtype: object

In [22]:
# Создаем датафреймы из данных
X=pd.DataFrame(df_train_prep, columns=features_without_categ)
X.head()

,Annual Income,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Bankruptcies,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
0,482087.0,0.0,11.0,26.3,685960.0,1.0,1.0,NaN,47386.0,7914.0,749.0
1,1025487.0,0.0,15.0,15.3,1181730.0,0.0,0.0,264968.0,394972.0,18373.0,737.0
2,751412.0,0.0,11.0,35.0,1182434.0,0.0,0.0,NaN,308389.0,13651.0,742.0
3,805068.0,0.0,8.0,22.5,147400.0,1.0,1.0,121396.0,95855.0,11338.0,694.0
4,776264.0,0.0,13.0,13.6,385836.0,1.0,0.0,125840.0,93309.0,7180.0,719.0


In [23]:
y = df_train_prep['Credit Default']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=True,stratify=df_train_prep['Credit Default'], random_state=2)
X_train.shape, X_test.shape

((5997, 11), (1500, 11))

In [25]:
def balance_df_by_target(df, target_name):

    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    return df.sample(frac=1) 

In [26]:
df_for_balancing = pd.concat([X_train, y_train], axis=1)
df_balanced = balance_df_by_target(df_for_balancing, target)
    
df_balanced[target].value_counts()

0    4307
1    3380
Name: Credit Default, dtype: int64

In [27]:
X_train = df_balanced.drop(columns=target)
y_train = df_balanced[target]

In [28]:
import catboost as cb
cb_params = {
    "n_estimators": 2000,
    "loss_function": "Logloss",
    "eval_metric": "F1",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 3,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42,
    #"cat_features": features_categ,
    #"reg_lambda":0.5,
    #'class_weights':[1, disbalance]
    
}

### - обучить любой классификатор (какой вам нравится)

In [29]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])

0:	learn: 0.5702006	test: 0.5702006	test1: 0.4495413	best: 0.4495413 (0)	total: 62.6ms	remaining: 2m 5s
10:	learn: 0.5833079	test: 0.5833079	test1: 0.4505263	best: 0.4505263 (10)	total: 137ms	remaining: 24.8s
20:	learn: 0.6004535	test: 0.6004535	test1: 0.4546403	best: 0.4634656 (14)	total: 207ms	remaining: 19.5s
30:	learn: 0.6037963	test: 0.6037963	test1: 0.4458465	best: 0.4634656 (14)	total: 282ms	remaining: 17.9s
40:	learn: 0.6088142	test: 0.6088142	test1: 0.4425349	best: 0.4634656 (14)	total: 354ms	remaining: 16.9s
50:	learn: 0.6126782	test: 0.6126782	test1: 0.4368308	best: 0.4634656 (14)	total: 425ms	remaining: 16.2s
60:	learn: 0.6149154	test: 0.6149154	test1: 0.4461371	best: 0.4634656 (14)	total: 496ms	remaining: 15.8s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4634655532
bestIteration = 14

Shrink model to first 15 iterations.


 ### - разобраться с SHAP и построить важности признаков для:

##### a. - всего тестового набора данных (summary_plot - дать интерпретацию)

In [30]:
import shap

ModuleNotFoundError: No module named 'shap'

In [34]:
pip install cuda

  ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install shap

  Using cached https://files.pythonhosted.org/packages/b9/f4/c5b95cddae15be80f8e58b25edceca105aa83c0b8c86a1edad24a6af80d3/shap-0.39.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/24/66/4720b6f70b42c74f10296a9803f8ba28c284f55cee6839f457bc67588277/numba-0.54.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/07/3c/bf72ebd3e78eb1ef773f4f0650ecdc29c6454aeafe9c08f6da3f227dd2bc/cloudpickle-2.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/55/21/f7df5d35f3f5d0637d64a89f6b0461f2adf78e22916d6372486f8fc2193d/llvmlite-0.37.0.tar.gz
  ERROR: Complete output from command /snap/jupyter/6/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-zlj47otj/shap/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-4xcl1449 --python-tag cp37:
  ERROR: The nvcc binary could not be

  ERROR: Complete output from command /snap/jupyter/6/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-zlj47otj/numba/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-uiy0sha2 --python-tag cp37:
  ERROR: TBB not found
  Using OpenMP from: True
  running bdist_wheel
  running build
  got version from file /tmp/pip-install-zlj47otj/numba/numba/_version.py {'version': '0.54.0', 'full': '5888895d3e893632f49334e15847703a06313f40'}
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/numba
  copying numba/runtests.py -> build/lib.linux-x86_64-3.7/numba
  copying numba/__main__.py -> build/lib.linux-x86_64-3.7/numba
  copying numba/__init__.py -> build/lib.linux-x86_64-3.7/numba
  copying numba/extending.py -> build/lib.linux-x86_64-3.7/numba
  copying n

  ERROR: Complete output from command /snap/jupyter/6/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-zlj47otj/llvmlite/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-93tnpbjj --python-tag cp37:
  ERROR: running bdist_wheel
  /snap/jupyter/6/bin/python /tmp/pip-install-zlj47otj/llvmlite/ffi/build.py
  LLVM version... Traceback (most recent call last):
    File "/tmp/pip-install-zlj47otj/llvmlite/ffi/build.py", line 220, in <module>
      main()
    File "/tmp/pip-install-zlj47otj/llvmlite/ffi/build.py", line 210, in main
      main_posix('linux', '.so')
    File "/tmp/pip-install-zlj47otj/llvmlite/ffi/build.py", line 134, in main_posix
      raise RuntimeError(msg) from None
  RuntimeError: Could not find a `llvm-config` binary. There are a number of reasons this could occur, please see: https://llvmli

In [ ]:
# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])